In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import cv2
from skimage import filters, feature
import matplotlib.pyplot as plt
from PIL import Image 
from tensorflow.keras.applications import EfficientNetB0
import os
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:

from tqdm import tqdm

# Input/Output directories
dataset_path = r"E:\plant dieaseas"
train_dir = os.path.join(dataset_path, "train")
val_dir = os.path.join(dataset_path, "validation")
test_dir = os.path.join(dataset_path, "test")

mask_base_dir = os.path.join(dataset_path, "masks")
os.makedirs(mask_base_dir, exist_ok=True)

IMG_SIZE = 256

def generate_leaf_mask(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"[ERROR] Could not read image: {image_path}")
        return None  # Skip this image

    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower = np.array([20, 40, 40])
    upper = np.array([85, 255, 255])
    mask = cv2.inRange(hsv, lower, upper)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest = max(contours, key=cv2.contourArea)
        clean_mask = np.zeros_like(mask)
        cv2.drawContours(clean_mask, [largest], -1, 255, thickness=cv2.FILLED)
        return clean_mask

    return mask


# Generate masks for entire dataset
def generate_masks_for_dir(image_dir, save_mask_dir):
    os.makedirs(save_mask_dir, exist_ok=True)
    for class_folder in os.listdir(image_dir):
        class_input_dir = os.path.join(image_dir, class_folder)
        class_mask_dir = os.path.join(save_mask_dir, class_folder)
        os.makedirs(class_mask_dir, exist_ok=True)

        for filename in tqdm(os.listdir(class_input_dir), desc=class_folder):
            # Skip non-image files
            if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            img_path = os.path.join(class_input_dir, filename)
            mask = generate_leaf_mask(img_path)

            if mask is not None:
                mask_path = os.path.join(class_mask_dir, filename)
                cv2.imwrite(mask_path, mask)


# Generate masks
generate_masks_for_dir(train_dir, os.path.join(mask_base_dir, "train"))
generate_masks_for_dir(val_dir, os.path.join(mask_base_dir, "validation"))
generate_masks_for_dir(test_dir, os.path.join(mask_base_dir, "test"))


Scab: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 35.01it/s]


In [3]:
import os
import cv2
import numpy as np

# Parameters
IMG_SIZE = 256
BATCH_SIZE = 16
EPOCHS = 20

# Base directories
base_dir = r"E:\plant dieaseas"
image_train_dir = os.path.join(base_dir, "train")
image_val_dir = os.path.join(base_dir, "validation")
image_test_dir = os.path.join(base_dir, "test")

mask_base_dir = os.path.join(base_dir, "masks")
mask_train_dir = os.path.join(mask_base_dir, "train")
mask_val_dir = os.path.join(mask_base_dir, "validation")
mask_test_dir = os.path.join(mask_base_dir, "test")

# Your function
def load_images_and_masks(image_dir, mask_dir, img_size):
    images = []
    masks = []
    for class_folder in os.listdir(image_dir):
        img_class_path = os.path.join(image_dir, class_folder)
        mask_class_path = os.path.join(mask_dir, class_folder)

        if not os.path.isdir(img_class_path) or not os.path.isdir(mask_class_path):
            continue  # skip non-folder entries

        for filename in os.listdir(img_class_path):
            if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue  # skip non-image files

            img_path = os.path.join(img_class_path, filename)
            mask_path = os.path.join(mask_class_path, filename)

            if not os.path.exists(mask_path):
                print(f"[WARNING] No mask for: {img_path}")
                continue

            image = cv2.imread(img_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            if image is None:
                print(f"[ERROR] Could not read image: {img_path}")
                continue
            if mask is None:
                print(f"[ERROR] Could not read mask: {mask_path}")
                continue

            image = cv2.resize(image, (img_size, img_size)) / 255.0
            mask = cv2.resize(mask, (img_size, img_size)) / 255.0
            mask = np.expand_dims(mask, axis=-1)

            images.append(image)
            masks.append(mask)

    return np.array(images), np.array(masks)


# Load datasets
train_images, train_masks = load_images_and_masks(image_train_dir, mask_train_dir, IMG_SIZE)
val_images, val_masks = load_images_and_masks(image_val_dir, mask_val_dir, IMG_SIZE)
test_images, test_masks = load_images_and_masks(image_test_dir, mask_test_dir, IMG_SIZE)

print(f"Train: {train_images.shape}, {train_masks.shape}")
print(f"Validation: {val_images.shape}, {val_masks.shape}")
print(f"Test: {test_images.shape}, {test_masks.shape}")


Train: (1940, 256, 256, 3), (1940, 256, 256, 1)
Validation: (160, 256, 256, 3), (160, 256, 256, 1)
Test: (400, 256, 256, 3), (400, 256, 256, 1)


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_ultra_light_unet(input_shape=(128, 128, 3)):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(16, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(16, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D()(c1)

    c2 = layers.Conv2D(32, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(32, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D()(c2)

    # Bottleneck
    c3 = layers.Conv2D(64, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(64, 3, activation='relu', padding='same')(c3)

    # Decoder
    u4 = layers.Conv2DTranspose(32, 2, strides=2, padding='same')(c3)
    u4 = layers.concatenate([u4, c2])
    c4 = layers.Conv2D(32, 3, activation='relu', padding='same')(u4)
    c4 = layers.Conv2D(32, 3, activation='relu', padding='same')(c4)

    u5 = layers.Conv2DTranspose(16, 2, strides=2, padding='same')(c4)
    u5 = layers.concatenate([u5, c1])
    c5 = layers.Conv2D(16, 3, activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(16, 3, activation='relu', padding='same')(c5)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c5)

    model = models.Model(inputs, outputs)
    return model


In [5]:
model = build_ultra_light_unet(input_shape=(128, 128, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# If needed: resize your data
train_images_resized = tf.image.resize(train_images, (128, 128))
train_masks_resized = tf.image.resize(train_masks, (128, 128))
val_images_resized = tf.image.resize(val_images, (128, 128))
val_masks_resized = tf.image.resize(val_masks, (128, 128))

# Train
model.fit(train_images_resized, train_masks_resized,
          validation_data=(val_images_resized, val_masks_resized),
          epochs=5, batch_size=8)


Epoch 1/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 62s 183ms/step - accuracy: 0.7752 - loss: 0.3476 - val_accuracy: 0.8904 - val_loss: 0.1539
Epoch 2/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 42s 174ms/step - accuracy: 0.8946 - loss: 0.1222 - val_accuracy: 0.9001 - val_loss: 0.1081
Epoch 3/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 43s 176ms/step - accuracy: 0.8960 - loss: 0.1101 - val_accuracy: 0.8981 - val_loss: 0.1029
Epoch 4/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 44s 180ms/step - accuracy: 0.9009 - loss: 0.0869 - val_accuracy: 0.9027 - val_loss: 0.0977
Epoch 5/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 35s 146ms/step - accuracy: 0.8999 - loss: 0.0919 - val_accuracy: 0.9013 - val_loss: 0.0886


In [6]:
model.save("unet_segmentation_model1.keras")

In [8]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Step 1: Load raw datasets
raw_train_ds = image_dataset_from_directory("E:/plant dieaseas/train", image_size=(224, 224), batch_size=32, label_mode='categorical')
raw_val_ds = image_dataset_from_directory("E:/plant dieaseas/validation", image_size=(224, 224), batch_size=32, label_mode='categorical')
raw_test_ds = image_dataset_from_directory("E:/plant dieaseas/test", image_size=(224, 224), batch_size=32, label_mode='categorical')

class_names = raw_train_ds.class_names
num_classes = len(class_names)

# Step 2: Data augmentation + normalization
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = raw_train_ds.map(lambda x, y: (normalization_layer(data_augmentation(x)), y))
val_ds = raw_val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = raw_test_ds.map(lambda x, y: (normalization_layer(x), y))

# ✅ Step 3: Add prefetch to datasets
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

# Step 4: Define and partially unfreeze DenseNet121
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Step 5: Build model
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=True)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs, outputs)

# Step 6: Compile model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 7: Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_densenet121_model_finetuned.h5", save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Step 8: Train model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    callbacks=[early_stop, checkpoint, reduce_lr]
)

# Step 9: Evaluate
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc:.2f}")


Found 1940 files belonging to 8 classes.
Found 160 files belonging to 8 classes.
Found 400 files belonging to 8 classes.
Epoch 1/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1941 - loss: 2.2767

61/61 ━━━━━━━━━━━━━━━━━━━━ 184s 3s/step - accuracy: 0.1948 - loss: 2.2736 - val_accuracy: 0.5938 - val_loss: 1.6870 - learning_rate: 1.0000e-04
Epoch 2/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4074 - loss: 1.6075

61/61 ━━━━━━━━━━━━━━━━━━━━ 161s 3s/step - accuracy: 0.4079 - loss: 1.6067 - val_accuracy: 0.7000 - val_loss: 1.1895 - learning_rate: 1.0000e-04
Epoch 3/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5498 - loss: 1.2907

61/61 ━━━━━━━━━━━━━━━━━━━━ 159s 3s/step - accuracy: 0.5501 - loss: 1.2900 - val_accuracy: 0.7688 - val_loss: 0.7810 - learning_rate: 1.0000e-04
Epoch 4/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6264 - loss: 1.0676

61/61 ━━━━━━━━━━━━━━━━━━━━ 179s 3s/step - accuracy: 0.6267 - loss: 1.0669 - val_accuracy: 0.8375 - val_loss: 0.5401 - learning_rate: 1.0000e-04
Epoch 5/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7369 - loss: 0.7884

...

61/61 ━━━━━━━━━━━━━━━━━━━━ 157s 3s/step - accuracy: 0.9053 - loss: 0.2681 - val_accuracy: 0.9500 - val_loss: 0.1462 - learning_rate: 1.0000e-04
Epoch 12/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 156s 3s/step - accuracy: 0.9336 - loss: 0.2118 - val_accuracy: 0.9375 - val_loss: 0.1607 - learning_rate: 1.0000e-04
Epoch 13/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9217 - loss: 0.2379

61/61 ━━━━━━━━━━━━━━━━━━━━ 155s 3s/step - accuracy: 0.9218 - loss: 0.2376 - val_accuracy: 0.9688 - val_loss: 0.1200 - learning_rate: 1.0000e-04
Epoch 14/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.9420 - loss: 0.1957 - val_accuracy: 0.9688 - val_loss: 0.1229 - learning_rate: 1.0000e-04
Epoch 15/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9434 - loss: 0.1672

61/61 ━━━━━━━━━━━━━━━━━━━━ 154s 3s/step - accuracy: 0.9433 - loss: 0.1675 - val_accuracy: 0.9688 - val_loss: 0.1029 - learning_rate: 1.0000e-04
Epoch 16/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9569 - loss: 0.1511

61/61 ━━━━━━━━━━━━━━━━━━━━ 171s 3s/step - accuracy: 0.9569 - loss: 0.1511 - val_accuracy: 0.9688 - val_loss: 0.0894 - learning_rate: 1.0000e-04
Epoch 17/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 178s 3s/step - accuracy: 0.9462 - loss: 0.1864 - val_accuracy: 0.9812 - val_loss: 0.0902 - learning_rate: 1.0000e-04
Epoch 18/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 176s 3s/step - accuracy: 0.9540 - loss: 0.1456 - val_accuracy: 0.9625 - val_loss: 0.1081 - learning_rate: 1.0000e-04
Epoch 19/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 178s 3s/step - accuracy: 0.9603 - loss: 0.1209 - val_accuracy: 0.9625 - val_loss: 0.0895 - learning_rate: 1.0000e-04
Epoch 20/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9668 - loss: 0.1105

61/61 ━━━━━━━━━━━━━━━━━━━━ 164s 3s/step - accuracy: 0.9667 - loss: 0.1106 - val_accuracy: 0.9750 - val_loss: 0.0824 - learning_rate: 5.0000e-05
Epoch 21/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 162s 3s/step - accuracy: 0.9670 - loss: 0.0903 - val_accuracy: 0.9688 - val_loss: 0.0874 - learning_rate: 5.0000e-05
Epoch 22/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 155s 3s/step - accuracy: 0.9656 - loss: 0.1013 - val_accuracy: 0.9688 - val_loss: 0.0942 - learning_rate: 5.0000e-05
Epoch 23/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 152s 2s/step - accuracy: 0.9733 - loss: 0.0890 - val_accuracy: 0.9625 - val_loss: 0.1000 - learning_rate: 5.0000e-05
Epoch 24/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 154s 3s/step - accuracy: 0.9696 - loss: 0.0854 - val_accuracy: 0.9688 - val_loss: 0.1037 - learning_rate: 2.5000e-05
Epoch 25/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 153s 3s/step - accuracy: 0.9762 - loss: 0.0847 - val_accuracy: 0.9688 - val_loss: 0.0970 - learning_rate: 2.5000e-05
13/13 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.9673 - loss: 0.1154
Tes